SPOTIFY MUSIC PREFERENCE ANALYSIS USING KNN

In [1]:
import requests
import json
import pandas as pd
import seaborn as sea
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, mean_squared_error

import timeit

CLIENT_ID = '9444e5d0012f4b2688a29c093833655c'
CLIENT_SECRET = '87fe2061e440456fa5dafed6388b672d'

In [2]:
#Sending the authentication token request using the auth_url
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [4]:
#Opening the csv file that contains details about my music library
with open('YourLibrary.json') as f:
    df = json.load(f)
    
df_library = df["tracks"]

In [5]:
#All the data is in a json format which has to be formatted
df_library

[{'artist': 'Santhosh Narayanan',
  'album': 'Madras',
  'track': 'Agayam Theepiditha',
  'uri': 'spotify:track:19nEFsswIftvopxVMUAM9b'},
 {'artist': 'Sid Sriram',
  'album': 'Enai Noki Paayum Thota (Original Motion Picture Soundtrack)',
  'track': 'Visiri',
  'uri': 'spotify:track:5Mt9VMVJc2T5Pa3qKUfw9t'},
 {'artist': 'S. P. Balasubrahmanyam',
  'album': 'Chandramukhi',
  'track': 'Athithom',
  'uri': 'spotify:track:2NeyA5MOLBZW21zKHBC2G2'},
 {'artist': 'G. V. Prakash',
  'album': 'Aayirathil Oruvan (Original Motion Picture Soundtrack)',
  'track': 'Maalai Neram',
  'uri': 'spotify:track:3S4ELYAcxunw60elgJwAwn'},
 {'artist': 'Harris Jayaraj',
  'album': 'The Legend',
  'track': 'Kone Komaane',
  'uri': 'spotify:track:0pgcemvSQD0ZdZBDtZqY5o'},
 {'artist': 'Vicanes Jay',
  'album': 'Adiyeh Kirukki (Single)',
  'track': 'Adiyeh Kirukki',
  'uri': 'spotify:track:7kq5biqsSgrS156U5mpBp2'},
 {'artist': 'Alyssa Mendonsa',
  'album': 'Zindagi Na Milegi Dobara',
  'track': 'Khaabon Ke Parinday'

In [6]:
#Converting the json data into a dataframe
df_library_final = pd.DataFrame(df_library, columns = ['artist', 'album', 'track', 'uri'])

In [7]:
#Converted to Dataframe
df_library_final

,artist,album,track,uri
0,Santhosh Narayanan,Madras,Agayam Theepiditha,spotify:track:19nEFsswIftvopxVMUAM9b
1,Sid Sriram,Enai Noki Paayum Thota (Original Motion Pictur...,Visiri,spotify:track:5Mt9VMVJc2T5Pa3qKUfw9t
2,S. P. Balasubrahmanyam,Chandramukhi,Athithom,spotify:track:2NeyA5MOLBZW21zKHBC2G2
3,G. V. Prakash,Aayirathil Oruvan (Original Motion Picture Sou...,Maalai Neram,spotify:track:3S4ELYAcxunw60elgJwAwn
4,Harris Jayaraj,The Legend,Kone Komaane,spotify:track:0pgcemvSQD0ZdZBDtZqY5o
...,...,...,...,...
140,Yuvan Shankar Raja,Rasaathi Nenja,Rasaathi Nenja - Madras Gig Season 2,spotify:track:0H1Uhrit1CxZzGBgsAz4at
141,Giveon,TAKE TIME,Heartbreak Anniversary,spotify:track:3FAJ6O0NOHQV8Mc5Ri6ENp
142,Vishal-Shekhar,Hasee Toh Phasee (Original Motion Picture Soun...,Zehnaseeb,spotify:track:3xF15SqY32Mno3UUMBirhU
143,Yazin Nizar,Sathya,Yavvana,spotify:track:0GJdVNyC7AOSN5F7zp08s0


In [8]:
#splitting the id of the song from the uri
list_uri = df_library_final['uri'].str.split(':', expand = True)

In [9]:
#List of ids'
list_uri_final = list_uri[2]

In [10]:
list_uri_final = list_uri_final.to_list()

In [11]:
#Retrieving audio features using song ids'
BASE_URL = 'https://api.spotify.com/v1/'
track_dict = {}
#Loop through list_uri and fetch track characteristics
for list_item in list_uri_final:
    track_dict[list_item] = {'danceability' : '', 'energy': '', 'key': '', 'loudness': '','mode': '','speechiness': '','acousticness': '','instrumentalness': '','liveness': '','valence': '','tempo': '','id': '','analysis_url': '','duration_ms': '','time_signature': ''}
    track_id = str(list_item)
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    r = r.json()
    track_dict[list_item] = r
    #track_data = pd.DataFrame.append(r, ignore_index = True)


    




In [12]:
#Retrieved audio features of each song in json format
track_dict

{'19nEFsswIftvopxVMUAM9b': {'danceability': 0.514,
  'energy': 0.423,
  'key': 0,
  'loudness': -8.388,
  'mode': 1,
  'speechiness': 0.04,
  'acousticness': 0.87,
  'instrumentalness': 0,
  'liveness': 0.0785,
  'valence': 0.362,
  'tempo': 117.87,
  'type': 'audio_features',
  'id': '19nEFsswIftvopxVMUAM9b',
  'uri': 'spotify:track:19nEFsswIftvopxVMUAM9b',
  'track_href': 'https://api.spotify.com/v1/tracks/19nEFsswIftvopxVMUAM9b',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/19nEFsswIftvopxVMUAM9b',
  'duration_ms': 230063,
  'time_signature': 4},
 '5Mt9VMVJc2T5Pa3qKUfw9t': {'danceability': 0.589,
  'energy': 0.346,
  'key': 7,
  'loudness': -8.742,
  'mode': 1,
  'speechiness': 0.0302,
  'acousticness': 0.656,
  'instrumentalness': 3.29e-05,
  'liveness': 0.131,
  'valence': 0.54,
  'tempo': 114.843,
  'type': 'audio_features',
  'id': '5Mt9VMVJc2T5Pa3qKUfw9t',
  'uri': 'spotify:track:5Mt9VMVJc2T5Pa3qKUfw9t',
  'track_href': 'https://api.spotify.com/v1/tracks/5Mt9VMV

In [13]:
#Converting the dictionary to dataframe
track_data = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','id','analysis_url','duration_ms','time_signature'])
for key in track_dict:
    track_data.loc[len(track_data)] = track_dict[key]



In [14]:
track_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,analysis_url,duration_ms,time_signature
0,0.514,0.423,0,-8.388,1,0.0400,0.8700,0.000000,0.0785,0.362,117.870,19nEFsswIftvopxVMUAM9b,https://api.spotify.com/v1/audio-analysis/19nE...,230063,4
1,0.589,0.346,7,-8.742,1,0.0302,0.6560,0.000033,0.1310,0.540,114.843,5Mt9VMVJc2T5Pa3qKUfw9t,https://api.spotify.com/v1/audio-analysis/5Mt9...,311127,4
2,0.746,0.757,10,-3.415,1,0.1320,0.4510,0.000002,0.0646,0.811,140.896,2NeyA5MOLBZW21zKHBC2G2,https://api.spotify.com/v1/audio-analysis/2Ney...,275387,3
3,0.563,0.428,10,-10.570,1,0.0424,0.8260,0.000232,0.1030,0.378,71.904,3S4ELYAcxunw60elgJwAwn,https://api.spotify.com/v1/audio-analysis/3S4E...,356960,4
4,0.810,0.787,9,-6.637,0,0.0624,0.3320,0.000000,0.1070,0.552,102.026,0pgcemvSQD0ZdZBDtZqY5o,https://api.spotify.com/v1/audio-analysis/0pgc...,335199,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,0.618,0.544,6,-6.336,1,0.0475,0.5570,0.000000,0.0746,0.322,91.999,0H1Uhrit1CxZzGBgsAz4at,https://api.spotify.com/v1/audio-analysis/0H1U...,287045,4
141,0.449,0.465,0,-8.964,1,0.0791,0.5240,0.000001,0.3030,0.543,89.087,3FAJ6O0NOHQV8Mc5Ri6ENp,https://api.spotify.com/v1/audio-analysis/3FAJ...,198371,3
142,0.623,0.595,7,-6.342,1,0.0331,0.3610,0.000136,0.3530,0.418,78.029,3xF15SqY32Mno3UUMBirhU,https://api.spotify.com/v1/audio-analysis/3xF1...,217440,4
143,0.675,0.826,5,-5.481,0,0.0522,0.0793,0.000000,0.1100,0.507,149.905,0GJdVNyC7AOSN5F7zp08s0,https://api.spotify.com/v1/audio-analysis/0GJd...,244800,4


In [15]:
track_data.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,0.627269,0.579979,5.055172,-8.429917,0.648276,0.063100,0.390988,0.032687,0.148010,0.502274,115.384117,264885.862069,3.896552
std,0.130004,0.183084,3.366046,2.712582,0.479164,0.056117,0.287035,0.134200,0.098197,0.221177,28.119447,78202.956842,0.385936
min,0.226000,0.110000,0.000000,-14.407000,0.000000,0.025500,0.001050,0.000000,0.037400,0.075800,64.934000,70500.000000,1.000000
25%,0.556000,0.449000,2.000000,-10.109000,0.000000,0.032700,0.131000,0.000000,0.086300,0.323000,91.991000,217440.000000,4.000000
50%,0.640000,0.567000,5.000000,-8.594000,1.000000,0.041900,0.357000,0.000036,0.110000,0.507000,114.934000,256500.000000,4.000000
75%,0.722000,0.717000,8.000000,-6.658000,1.000000,0.060500,0.635000,0.001010,0.169000,0.659000,130.002000,306080.000000,4.000000
max,0.905000,0.984000,11.000000,-2.479000,1.000000,0.375000,0.946000,0.908000,0.536000,0.965000,199.671000,699261.000000,5.000000


#We'll will now try to retrieve playlist data involving Blake Shelton's music since it's country music that I don't normally listen to. (Done to have negative datasets)

In [16]:
#links to retrieve blake shelton's album(trial attempt)
country_base_url = 'https://api.spotify.com/v1/tracks/'
gods_country_id = '1dyxS3YzBbL0t8rEVwVQ1K?si=99ec397592f84ad1'

In [17]:
print(country_base_url + gods_country_id)

https://api.spotify.com/v1/tracks/1dyxS3YzBbL0t8rEVwVQ1K?si=99ec397592f84ad1


In [18]:
#Sending request to retrieve album data
p = requests.get(country_base_url + gods_country_id, headers=headers)
p = p.json()

In [19]:
#Retrieving blake shelton's artist id to pull tracks sung by him
blake_shelton_id = p['artists'][0]['id']

In [20]:
artist_id = blake_shelton_id

# pull all artists albums
a = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
a = a.json()

In [21]:
#Pulling all albums of blake shelton
for album in a['items']:
    print(album['name'], ' --- ', album['release_date'])

Cheers, It's Christmas (Super Deluxe)  ---  2022-11-11
Body Language (Deluxe)  ---  2021-12-03
Body Language  ---  2021-05-21
Fully Loaded: God's Country  ---  2019-12-13
Texoma Shore  ---  2017-11-03
If I'm Honest  ---  2016-05-20
BRINGING BACK THE SUNSHINE  ---  2014-09-30
Based on a True Story... (Deluxe Edition)  ---  2013-03-26
Cheers, It's Christmas (Deluxe Edition)  ---  2012-10-02
Red River Blue (Deluxe Edition)  ---  2011-07-11
Startin' Fires  ---  2008-11-10
Pure BS (Deluxe Edition)  ---  2007-05-01
Blake Shelton's Barn & Grill  ---  2004-10-26
The Dreamer  ---  2003
Blake Shelton  ---  2001-07-31


In [22]:
for album in a['items']:
    album_name = album['name']

In [23]:
#Pulling all tracks in that specific album
a = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', headers=headers)
tracks = a.json()['items']

In [24]:
tracks

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1UTPBmNbXNTittyMJrNkvw'},
    'href': 'https://api.spotify.com/v1/artists/1UTPBmNbXNTittyMJrNkvw',
    'id': '1UTPBmNbXNTittyMJrNkvw',
    'name': 'Blake Shelton',
    'type': 'artist',
    'uri': 'spotify:artist:1UTPBmNbXNTittyMJrNkvw'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   'KW',
   'EG',
   'MA',
   'D

In [25]:
data = []   # will hold all track info
albums = [] # to keep track of duplicates
for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(BASE_URL + 'audio-features/' + track['id'], 
            headers=headers)
        f = f.json()
        
        # combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
        
        data.append(f)

In [26]:
#Converting track data to dataframe
blake_shelton_track_data = pd.DataFrame(data)

In [27]:
#Adding classifier to that track data (0 - negative class, 1- positive class)
blake_shelton_track_data['likeLabel'] = 0

In [28]:
#Removing unnecessary data from dataframe
blake_shelton_track_data = blake_shelton_track_data.drop(['album_name','release_date','track_name','track_href','uri','album_id','type'], axis = 1)

IMPORTING COUNTRY DATA OBTAINED DURING A DIFFERENT DATA PULL

In [29]:
#Importing country music(more sample data to better train the model)
country_track_data = pd.read_csv('/Users/aswinyay/Downloads/trackdata.csv')

In [30]:
#Dropping unncessary columns
country_track_data = country_track_data.drop(['track_href','type','uri','Unnamed: 0'], axis = 1)

In [31]:
#Adding likelabels(Again, since it's country music, the label is negative class)
country_track_data['likeLabel'] = 0

In [32]:
country_track_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,analysis_url,duration_ms,time_signature,likeLabel
0,0.566,0.713,6,-4.250,1,0.0417,0.64500,0.000000,0.1140,0.807,169.864,4dGJf1SER1T6ooX46vwzRB,https://api.spotify.com/v1/audio-analysis/4dGJ...,238147,4,0
1,0.595,0.389,2,-9.245,1,0.0349,0.41800,0.000001,0.1160,0.400,166.052,5kNe7PE09d6Kvw5pAsx23n,https://api.spotify.com/v1/audio-analysis/5kNe...,227427,4,0
2,0.523,0.666,7,-4.081,0,0.0328,0.40100,0.000000,0.0726,0.562,78.075,2tUJbvjNznjNZfCeKtkhGm,https://api.spotify.com/v1/audio-analysis/2tUJ...,194760,4,0
3,0.607,0.536,10,-7.306,1,0.0305,0.38600,0.000000,0.1020,0.434,147.986,1DA7FEKgQeyRPtU44L5DOf,https://api.spotify.com/v1/audio-analysis/1DA7...,226200,4,0
4,0.661,0.750,7,-5.497,1,0.0314,0.03030,0.000132,0.0846,0.828,130.077,2JGUcpSLhxxruwkqQfBhAl,https://api.spotify.com/v1/audio-analysis/2JGU...,157573,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.651,0.782,8,-4.295,1,0.0718,0.29500,0.000000,0.1070,0.945,171.987,0OlS52TmtK2gPaEtdmr7DN,https://api.spotify.com/v1/audio-analysis/0OlS...,176893,4,0
96,0.612,0.848,6,-3.736,1,0.0368,0.00916,0.000000,0.3660,0.719,120.996,66pxmg3Jhjm09tHoWR1wVR,https://api.spotify.com/v1/audio-analysis/66px...,212893,4,0
97,0.626,0.913,0,-4.452,1,0.0318,0.00635,0.000000,0.2930,0.946,113.044,1cktZnxiONH0hR6Q3DKy8h,https://api.spotify.com/v1/audio-analysis/1ckt...,191307,4,0
98,0.592,0.891,10,-3.144,1,0.0512,0.02670,0.000000,0.3520,0.674,99.983,3dCZm5HrG0jIXAHfNkg2H0,https://api.spotify.com/v1/audio-analysis/3dCZ...,174255,4,0


In [33]:
#concatenating both the dataframes
blake_shelton_track_data = pd.concat([blake_shelton_track_data, country_track_data])

In [34]:
#Adding the like label to my data(1 since it's positive class)
track_data['likeLabel'] = 1

In [35]:
blake_shelton_track_data.groupby(by = 'likeLabel').count()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,analysis_url,duration_ms,time_signature
likeLabel,,,,,,,,,,,,,,,
0,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110


In [36]:
final_data_list = pd.concat([track_data,blake_shelton_track_data])

In [37]:
final_data_list.groupby(by = 'likeLabel').count()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,analysis_url,duration_ms,time_signature
likeLabel,,,,,,,,,,,,,,,
0,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110
1,145,145,145,145,145,145,145,145,145,145,145,145,145,145,145


KNN code

In [38]:
final_data_list.shape

(255, 16)

In [39]:
final_data_list.isnull().sum()

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
id                  0
analysis_url        0
duration_ms         0
time_signature      0
likeLabel           0
dtype: int64

In [40]:
final_data_list.groupby(by = 'likeLabel').count()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,analysis_url,duration_ms,time_signature
likeLabel,,,,,,,,,,,,,,,
0,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110
1,145,145,145,145,145,145,145,145,145,145,145,145,145,145,145


In [41]:
model_list = final_data_list[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature','likeLabel']]

In [42]:
model_list.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,likeLabel
0,0.514,0.423,0,-8.388,1,0.0400,0.8700,0.000000,0.0785,0.362,117.870,4,1
1,0.589,0.346,7,-8.742,1,0.0302,0.6560,0.000033,0.1310,0.540,114.843,4,1
2,0.746,0.757,10,-3.415,1,0.1320,0.4510,0.000002,0.0646,0.811,140.896,3,1
3,0.563,0.428,10,-10.570,1,0.0424,0.8260,0.000232,0.1030,0.378,71.904,4,1
4,0.810,0.787,9,-6.637,0,0.0624,0.3320,0.000000,0.1070,0.552,102.026,4,1
5,0.624,0.898,6,-4.827,0,0.0858,0.1790,0.000000,0.1240,0.787,131.973,4,1
6,0.517,0.565,9,-10.184,1,0.0441,0.7210,0.006890,0.0997,0.690,169.952,4,1
7,0.585,0.444,1,-9.969,0,0.0458,0.4120,0.000070,0.2420,0.233,119.774,5,1
8,0.595,0.832,4,-4.460,1,0.0400,0.0162,0.000000,0.0728,0.417,104.951,4,1
9,0.689,0.936,5,-4.481,0,0.0741,0.2170,0.391000,0.1890,0.193,125.989,4,1


In [43]:
predictors_df = model_list.loc[:,:'time_signature']
response_df = model_list.loc[:, 'likeLabel']

In [44]:
predictors_df.corr()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
danceability,1.000000,0.127868,-0.082328,-0.036185,-0.051720,0.224290,-0.252353,-0.162258,0.047683,0.445070,-0.157321,0.186200
energy,0.127868,1.000000,0.022109,0.744711,-0.007756,0.238355,-0.700322,-0.056626,0.141968,0.470127,0.159633,0.128502
key,-0.082328,0.022109,1.000000,0.000021,-0.045855,-0.120453,-0.060245,0.022810,-0.035057,-0.020569,0.002161,0.018351
loudness,-0.036185,0.744711,0.000021,1.000000,0.055145,0.099454,-0.528866,-0.191440,0.085102,0.314763,0.154159,0.050500
mode,-0.051720,-0.007756,-0.045855,0.055145,1.000000,-0.182764,-0.009952,-0.051742,0.003812,0.038150,0.037546,-0.069762
speechiness,0.224290,0.238355,-0.120453,0.099454,-0.182764,1.000000,-0.116727,0.070783,0.074309,0.227777,0.131494,-0.019165
acousticness,-0.252353,-0.700322,-0.060245,-0.528866,-0.009952,-0.116727,1.000000,0.138013,-0.175651,-0.371236,-0.070031,-0.140628
instrumentalness,-0.162258,-0.056626,0.022810,-0.191440,-0.051742,0.070783,0.138013,1.000000,-0.040713,-0.100199,-0.024994,0.004442
liveness,0.047683,0.141968,-0.035057,0.085102,0.003812,0.074309,-0.175651,-0.040713,1.000000,0.076379,0.042886,0.023639
valence,0.445070,0.470127,-0.020569,0.314763,0.038150,0.227777,-0.371236,-0.100199,0.076379,1.000000,0.117118,0.107416


In [45]:
predictors_df.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object

In [46]:
# partition data into train and test sets
X = predictors_df
Y = response_df
train_X_classifier, test_X_classifier, train_y_classifier, test_y_classifier = train_test_split(X, 
                                                            Y, test_size=0.4, random_state=1)

In [47]:
# normalize predictors in auctions data using standardization
z_score_norm1 = preprocessing.StandardScaler()
z_score_norm1.fit(train_X_classifier)
train_X_classifier = pd.DataFrame(z_score_norm1.transform(train_X_classifier), 
                                          columns = predictors_df.columns)
test_X_classifier = pd.DataFrame(z_score_norm1.transform(test_X_classifier), 
                                          columns = predictors_df.columns)
test_X_classifier

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,-1.018619,-0.180901,1.576743,0.657939,0.544619,-0.519732,-1.126892,-0.203656,-0.250898,-0.195131,-1.023288,0.240008
1,0.079141,-0.495989,-0.986628,-0.028999,0.544619,-0.376322,0.773334,-0.203563,1.943497,-1.324863,0.752214,0.240008
2,-2.166277,-1.756338,-0.416990,-0.961561,-1.836145,-0.406742,2.258119,-0.203656,-0.610157,-1.898840,-0.058213,0.240008
3,1.867160,-0.545739,1.007105,-2.260713,0.544619,-0.419779,-0.752696,-0.203346,-0.786874,1.476691,-0.265222,0.240008
4,-0.045604,-0.645241,-0.416990,0.332411,0.544619,-0.528424,0.060937,-0.203656,1.137591,-0.796440,0.451395,0.240008
...,...,...,...,...,...,...,...,...,...,...,...,...
97,-1.168313,1.565898,1.007105,1.938224,0.544619,0.166901,-0.520229,-0.203656,0.370524,0.829828,1.414854,0.240008
98,-0.344993,0.388467,0.152648,0.168537,0.544619,-0.119921,-0.733949,-0.203656,0.457911,-0.026582,-0.924499,0.240008
99,-0.261830,-0.402015,-0.416990,0.689382,-1.836145,-0.541461,0.447131,-0.203632,-0.357704,-0.564116,0.425972,-3.432110
100,1.002258,1.289506,-0.986628,1.327490,0.544619,-0.463237,-1.161012,-0.203656,0.856009,1.586020,-0.133130,0.240008


In [48]:
# train the k-NN model and look at performance on train data

knn = KNeighborsClassifier(n_neighbors=5).fit(train_X_classifier, train_y_classifier)
predicted_y_training = knn.predict(train_X_classifier)
f1_score(train_y_classifier, predicted_y_training)

/Users/aswinyay/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.8433734939759037

In [49]:
# performance of k-NN on test data

predicted_y_test = knn.predict(test_X_classifier)
f1_score(test_y_classifier, predicted_y_test)

/Users/aswinyay/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.8108108108108109

In [50]:
# train a classifier for different values of k

results = []
for k in range(1, 20):
    knn2 = KNeighborsClassifier(n_neighbors=k).fit(train_X_classifier, train_y_classifier)
    results.append({
        'k': k,
        'f1_score': f1_score(test_y_classifier, knn2.predict(test_X_classifier))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

     k  f1_score
0    1  0.722222
1    2  0.630435
2    3  0.796460
3    4  0.701031
4    5  0.810811
5    6  0.740000
6    7  0.825688
7    8  0.707071
8    9  0.796296
9   10  0.745098
10  11  0.814815
11  12  0.707071
12  13  0.742857
13  14  0.712871
14  15  0.742857
15  16  0.693069
16  17  0.718447
17  18  0.686869
18  19  0.725490


/Users/aswinyay/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/aswinyay/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no lo

In [53]:
user_tracks = track_data.to_csv('/Users/aswinyay/Downloads/user_data.csv')